# Python options when using Trino & Starburst

This notebook is focused on showing the primary options available to Python programmers when using Starburst.

1.   [Python UDFs](https://trino.io/docs/current/udf/python.html)
2.   [Python client](https://github.com/trinodb/trino-python-client)
3.   [PyStarburst](https://docs.starburst.io/clients/python/pystarburst.html) (ONLY available with Starburst)
4.   [Ibis](https://ibis-project.org/)



## Trino Python user-defined functions

TODO - ramble about how this is avail with Trino, but for Starburst it is only on SEP and even there, as a public preview (not on Galaxy)

DON'T TRY TO RUN HERE... GIVE EXAMPLES OF SQL UDF AND PORTS TO PYTHON UDF (and tell them to run in Starburst UI (or Trino CLI))

In [5]:
import getpass

# grab credentials from the notebook user to be used when making a connection
my_host = input("Host name")
my_username = input("User name")
my_password = getpass.getpass("Password")



Host namelester-aws-us-east-1-free.trino.galaxy.starburst.io
User namelester.martin@starburstdata.com/accountadmin
Password··········


## Trino Python client

TODO - RAMLBE...


In [3]:
pip install trino

In [20]:
from trino.dbapi import connect
from trino.auth import BasicAuthentication

# sanity check
print('\n Make sure the phrase ** CONNECTION IS GOOD ** displays \n')


# build the connection object with the hostname & creds inputed earlier
conn = connect(
    host=my_host,
    port="443",
    user=my_username,
    auth=BasicAuthentication(my_username, my_password),
    http_scheme="https",
    catalog="system",
    schema="runtime",
)
cur = conn.cursor()
cur.execute("SELECT '** CONNECTION IS GOOD **'")
rows = cur.fetchall()
print(rows)


 Make sure the phrase ** CONNECTION IS GOOD ** displays 

[['** CONNECTION IS GOOD **']]


In [30]:
import pandas as pd


cur = conn.cursor()
cur.execute("SELECT * FROM tpch.tiny.nation")
rows = cur.fetchall()
col_name = [desc[0] for desc in cur.description]
pdf = pd.DataFrame(rows, columns=col_name)

pdf




,nationkey,name,regionkey,comment
0,0,ALGERIA,0,haggle. carefully final deposits detect slyly...
1,1,ARGENTINA,1,al foxes promise slyly according to the regula...
2,2,BRAZIL,1,y alongside of the pending deposits. carefully...
3,3,CANADA,1,"eas hang ironic, silent packages. slyly regula..."
4,4,EGYPT,4,y above the carefully unusual theodolites. fin...
5,5,ETHIOPIA,0,ven packages wake quickly. regu
6,6,FRANCE,3,"refully final requests. regular, ironi"
7,7,GERMANY,3,"l platelets. regular accounts x-ray: unusual, ..."
8,8,INDIA,2,ss excuses cajole slyly across the packages. d...
9,9,INDONESIA,2,slyly express asymptotes. regular deposits ha...


## PyStarburst

TODO - RAMLBE...

In [28]:
import trino

from pystarburst import Session
from pystarburst import functions as F
from pystarburst.functions import *
from pystarburst.window import Window as W

# PyStarburst setup
session_properties = {
    "host":my_host,
    "port": 443,
    "http_scheme": "https",
    "auth": trino.auth.BasicAuthentication(my_username, my_password)
}
session = Session.builder.configs(session_properties).create()

# validate PyStarburst working
print('\n Make sure the phrase ** CONNECTION IS GOOD ** displays \n')
session.sql("select '** CONNECTION IS GOOD **' as conn_check").collect()


 Make sure the phrase ** CONNECTION IS GOOD ** displays 



[Row(conn_check='** CONNECTION IS GOOD **')]

In [32]:
# Example 1 (PyStarburst): For each nation, get top X (rn <= X) customers by acctbal.
# Style: DataFrame API modeled after PySpark; also supports session.sql("...")
# Tables: tpch.tiny.customer, tpch.tiny.nation

# Fully qualified names are explicit and avoid relying on default catalog/schema

from pystarburst.window import Window

customer = session.table(f"tpch.tiny.customer")
nation = session.table(f"tpch.tiny.nation") \
            .drop("regionkey", "comment") \
            .rename("name", "nation_name") \
            .rename("nationkey", "n_nationkey")

filtered = customer.select("custkey", "name", "acctbal", "nationkey") \
            .filter(col("acctbal") > 8000.0)
joined = filtered.join(nation, col("nationkey") == nation.n_nationkey) \
            .drop("nationkey", "n_nationkey")

w = Window.partitionBy("nation_name").orderBy(col("acctbal").desc())
ranked = joined.select("*", row_number().over(w).alias("rn"))
top_x = ranked.filter(col("rn") <= 1).sort(col("acctbal").desc(), col("nation_name"))
top_x.show(25)

----------------------------------------------------------------------
|"custkey"  |"name"              |"acctbal"  |"nation_name"   |"rn"  |
----------------------------------------------------------------------
|213        |Customer#000000213  |9987.71    |UNITED STATES   |1     |
|45         |Customer#000000045  |9983.38    |INDONESIA       |1     |
|1106       |Customer#000001106  |9977.62    |VIETNAM         |1     |
|200        |Customer#000000200  |9967.6     |MOZAMBIQUE      |1     |
|140        |Customer#000000140  |9963.15    |EGYPT           |1     |
|381        |Customer#000000381  |9931.71    |ETHIOPIA        |1     |
|43         |Customer#000000043  |9904.28    |ROMANIA         |1     |
|100        |Customer#000000100  |9889.89    |SAUDI ARABIA    |1     |
|780        |Customer#000000780  |9874.12    |INDIA           |1     |
|518        |Customer#000000518  |9871.66    |PERU            |1     |
|197        |Customer#000000197  |9860.22    |ARGENTINA       |1     |
|219  

## Ibis

TODO - RAMLBE...

In [ ]:
#
# Install the library
#

%pip install trino
%pip install 'ibis-framework[trino]'
%pip install pystarburst

In [43]:
import os
import ibis
from trino.auth import BasicAuthentication

ibis.options.interactive = True


user = my_username
trino_auth_obj = BasicAuthentication(my_username, my_password)
host = my_host
port = "443"
http_scheme = "https"
catalog = "tpch"
schema = "tiny"

con = ibis.trino.connect(
    user=user, auth=trino_auth_obj, host=host, port=port, http_scheme=http_scheme, database=catalog, schema=schema
)


print('\n Make sure the phrase ** CONNECTION IS GOOD ** displays \n')
con.sql("select '** CONNECTION IS GOOD **' as conn_check")




 Make sure the phrase ** CONNECTION IS GOOD ** displays 



┏━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ conn_check               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                   │
├──────────────────────────┤
│ ** CONNECTION IS GOOD ** │
└──────────────────────────┘

In [47]:
nations = con.table("nation")
nations[0:50]

┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ nationkey ┃ name      ┃ regionkey ┃ comment                                                                          ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !int64    │ !string   │ !int64    │ !string                                                                          │
├───────────┼───────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┤
│         0 │ ALGERIA   │         0 │  haggle. carefully final deposits detect slyly agai                              │
│         1 │ ARGENTINA │         1 │ al foxes promise slyly according to the regular accounts. bold requests alon     │
│         2 │ BRAZIL    │         1 │ y alongside of the pending deposits. carefully special packages are about the i… │
│         3 │ CANADA    │         1 │ eas hang ironic, silent packages. slyly regular packages are furiously over the… │
│         4 │ EGYPT     │         4 │ y above the carefully unusual theodolites. final dugouts are quickly across the… │
│         5 │ ETHIOPIA  │         0 │ ven packages wake quickly. regu                                                  │
│         6 │ FRANCE    │         3 │ refully final requests. regular, ironi                                           │
│         7 │ GERMANY   │         3 │ l platelets. regular accounts x-ray: unusual, regular acco                       │
│         8 │ INDIA     │         2 │ ss excuses cajole slyly across the packages. deposits print aroun                │
│         9 │ INDONESIA │         2 │  slyly express asymptotes. regular deposits haggle slyly. carefully ironic hock… │
│         … │ …         │         … │ …                                                                                │
└───────────┴───────────┴───────────┴──────────────────────────────────────────────────────────────────────────────────┘